In [23]:
import sys
sys.path.append('../scripts')
import polars as pl
from helpers import load_master_dataset

# Load master dataset and filter by source file
source_file = "orig_adc_record.6"
df = load_master_dataset('../../usb_master_dataset.parquet')
df = df.filter(pl.col('source_file') == source_file)
bulk_df = df.filter(pl.col('transfer_type') != '0x02')

print(f"Source file: {source_file}")
print(f"Dataset: {len(df):,} packets")
print(f"Bulk transfers: {len(bulk_df):,} packets ({len(bulk_df)/len(df)*100:.1f}%)")
print(f"Control packets filtered: {len(df)-len(bulk_df)}")
print(f"\nColumn names ({len(df.columns)}):")
print(df.columns)

✅ Loaded 11,514 USB packets from ../../usb_master_dataset.parquet
Source file: orig_adc_record.6
Dataset: 420 packets
Bulk transfers: 394 packets (93.8%)
Control packets filtered: 26

Column names (40):
['session_id', 'frame_number', 'timestamp', 'timestamp_absolute', 'direction', 'device_address', 'bus_id', 'endpoint_address', 'endpoint_number', 'transfer_type', 'urb_type', 'urb_status', 'data_length', 'urb_length', 'payload_hex', 'setup_flag', 'data_flag', 'interval', 'start_frame', 'frame_length', 'frame_protocols', 'source_file', 'bmrequest_type', 'brequest', 'brequest_name', 'wvalue', 'windex', 'wlength', 'descriptor_type', 'descriptor_index', 'language_id', 'transfer_flags', 'copy_of_transfer_flags', 'urb_id', 'usb_src', 'usb_dst', 'usb_addr', 'urb_ts_sec', 'urb_ts_usec', 'added_datetime']


In [24]:
# Find first row with payload of 52 bytes
payload_52b = df.filter((pl.col('data_length') == 52) & (pl.col('payload_hex') != ''))

# Show key columns only
key_columns = ['timestamp', 'device_address', 'direction', 'endpoint_address', 'data_length', 'payload_hex']
print("First row with 52-byte payload (key columns):")
print(payload_52b.select(key_columns)[0])

# Show the payload hex data
print(f"\nPayload hex: {payload_52b[0, 'payload_hex']}")

# Show all columns transposed (column-by-column)
print(f"\nAll columns (transposed view):")
first_row = payload_52b[0]
for col_name in df.columns:
    value = first_row[col_name][0]  # Get the single value
    print(f"{col_name:20s}: {value}")


First row with 52-byte payload (key columns):
shape: (1, 6)
┌───────────┬────────────────┬───────────┬──────────────────┬─────────────┬────────────────────────┐
│ timestamp ┆ device_address ┆ direction ┆ endpoint_address ┆ data_length ┆ payload_hex            │
│ ---       ┆ ---            ┆ ---       ┆ ---              ┆ ---         ┆ ---                    │
│ f64       ┆ u32            ┆ str       ┆ str              ┆ u32         ┆ str                    │
╞═══════════╪════════════════╪═══════════╪══════════════════╪═════════════╪════════════════════════╡
│ 2.760307  ┆ 6              ┆ D->H      ┆ 0x81             ┆ 52          ┆ 410a82020100000b2d8e4d │
│           ┆                ┆           ┆                  ┆             ┆ 00e6ffff…              │
└───────────┴────────────────┴───────────┴──────────────────┴─────────────┴────────────────────────┘

Payload hex: 410a82020100000b2d8e4d00e6ffffff218e4d00eaffffff278e4d00480000001b0c9402747e000000007c7e0080a00c00000000

All columns 

In [12]:
# Basic bulk transfer statistics
print("Transfer Types:")
print(bulk_df['transfer_type'].value_counts())
print("\nDevices:")
print(bulk_df['device_address'].value_counts())
print("\nEndpoints:")
print(bulk_df['endpoint_address'].value_counts())
print("\nDirections:")
print(bulk_df['direction'].value_counts())


Transfer Types:
shape: (1, 2)
┌───────────────┬───────┐
│ transfer_type ┆ count │
│ ---           ┆ ---   │
│ str           ┆ u32   │
╞═══════════════╪═══════╡
│ 0x03          ┆ 394   │
└───────────────┴───────┘

Devices:
shape: (1, 2)
┌────────────────┬───────┐
│ device_address ┆ count │
│ ---            ┆ ---   │
│ u32            ┆ u32   │
╞════════════════╪═══════╡
│ 6              ┆ 394   │
└────────────────┴───────┘

Endpoints:
shape: (2, 2)
┌──────────────────┬───────┐
│ endpoint_address ┆ count │
│ ---              ┆ ---   │
│ str              ┆ u32   │
╞══════════════════╪═══════╡
│ 0x01             ┆ 190   │
│ 0x81             ┆ 204   │
└──────────────────┴───────┘

Directions:
shape: (2, 2)
┌───────────┬───────┐
│ direction ┆ count │
│ ---       ┆ ---   │
│ str       ┆ u32   │
╞═══════════╪═══════╡
│ D->H      ┆ 204   │
│ H->D      ┆ 190   │
└───────────┴───────┘


In [13]:
# Payload analysis
payload_stats = bulk_df.select([
    pl.col('data_length').mean().alias('avg_payload_size'),
    pl.col('data_length').max().alias('max_payload_size'),
    pl.col('data_length').min().alias('min_payload_size'),
    (pl.col('payload_hex') != '').sum().alias('packets_with_payload'),
    pl.len().alias('total_bulk_packets')
])
print("Payload Statistics:")
print(payload_stats)


Payload Statistics:
shape: (1, 5)
┌──────────────────┬──────────────────┬──────────────────┬────────────────────┬────────────────────┐
│ avg_payload_size ┆ max_payload_size ┆ min_payload_size ┆ packets_with_paylo ┆ total_bulk_packets │
│ ---              ┆ ---              ┆ ---              ┆ ad                 ┆ ---                │
│ f64              ┆ u32              ┆ u32              ┆ ---                ┆ u32                │
│                  ┆                  ┆                  ┆ u32                ┆                    │
╞══════════════════╪══════════════════╪══════════════════╪════════════════════╪════════════════════╡
│ 13.736041        ┆ 188              ┆ 0                ┆ 195                ┆ 394                │
└──────────────────┴──────────────────┴──────────────────┴────────────────────┴────────────────────┘


In [14]:
# Device breakdown
device_stats = bulk_df.group_by('device_address').agg([
    pl.len().alias('packet_count'),
    pl.col('data_length').mean().alias('avg_payload'),
    (pl.col('payload_hex') != '').sum().alias('with_payload')
])
print("Device Breakdown:")
print(device_stats)


Device Breakdown:
shape: (1, 4)
┌────────────────┬──────────────┬─────────────┬──────────────┐
│ device_address ┆ packet_count ┆ avg_payload ┆ with_payload │
│ ---            ┆ ---          ┆ ---         ┆ ---          │
│ u32            ┆ u32          ┆ f64         ┆ u32          │
╞════════════════╪══════════════╪═════════════╪══════════════╡
│ 6              ┆ 394          ┆ 13.736041   ┆ 195          │
└────────────────┴──────────────┴─────────────┴──────────────┘


In [15]:
# Payload size patterns by device
size_patterns = bulk_df.filter(pl.col('payload_hex') != '').group_by(['device_address', 'data_length']).agg([
    pl.len().alias('count')
])

print("Payload Size Patterns:")
current_device = None
for row in size_patterns.sort(['device_address', 'count'], descending=[False, True]).iter_rows(named=True):
    if row['device_address'] != current_device:
        current_device = row['device_address']
        print(f"\nDevice {current_device}:")
    if row['count'] > 10:
        print(f"  {row['data_length']}b: {row['count']} packets")


Payload Size Patterns:

Device 6:
  4b: 96 packets
  52b: 74 packets
